In [1]:
import os
os.environ['USE_PYGEOS'] = '0'
from urbantrips.datamodel import services
from urbantrips.datamodel import transactions as trx
from urbantrips.utils import utils
from datetime import datetime
from urbantrips.utils.check_configs import check_config

# def main():
if True:
    
    # # Check config file consistency
    check_config()

    # Read config file
    configs = utils.leer_configs_generales()

    # trx configs
    formato_fecha = configs["formato_fecha"]
    nombre_archivo_gps = configs["nombre_archivo_gps"]
    nombres_variables_gps = configs["nombres_variables_gps"]

    print("INICIO GPS")
    print(datetime.now())

    # SUBO LOS DATOS DE GPS , TRX YA GEOLOCALIZADO
    trx.process_and_upload_gps_table(
        nombre_archivo_gps=nombre_archivo_gps,
        nombres_variables_gps=nombres_variables_gps,
        formato_fecha=formato_fecha)

    print("FIN GPS")
    print(datetime.now())

    # proceso servicios
    print("INICIO SERVICIOS")
    print(datetime.now())
    services.process_services()
    print("FIN SERVICIOS")
    print(datetime.now())


# if __name__ == "__main__":
#     main()

Se concluyó el chequeo del archivo de configuración
INICIO GPS
2023-07-06 10:25:50.539062
Procesando tabla gps


OperationalError: database is locked

In [2]:
import pandas as pd
import sqlite3
import os
import yaml
import time
from functools import wraps
import h3
import numpy as np
import weightedstats as ws
from pandas.io.sql import DatabaseError

In [4]:
from datetime import datetime
import networkx as nx
import osmnx as ox
import pandas as pd
from pandas.io.sql import DatabaseError
import numpy as np
import itertools
import os
import geopandas as gpd
import h3
from networkx import NetworkXNoPath
import multiprocessing
from functools import partial
from math import sqrt
from urbantrips.geo.geo import (
    get_stop_hex_ring, h3togeo, add_geometry,
    create_voronoi, normalizo_lat_lon, h3dist, bring_latlon
)
from urbantrips.viz import viz
from urbantrips.utils.utils import (
    duracion,
    iniciar_conexion_db,
    leer_configs_generales,
    leer_alias)

C:\Users\sanap\miniconda3\envs\urbantrips\lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
C:\Users\sanap\miniconda3\envs\urbantrips\lib\site-packages\osmnx\projection.py:3: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (h

In [6]:
conn_insumos = iniciar_conexion_db(tipo='insumos')

distancias = pd.read_sql_query(
    """
    SELECT *
    FROM distancias
    """,
    conn_insumos,
)
distancias

,h3_o,h3_d,h3_o_norm,h3_d_norm,distance_osm_drive,distance_osm_walk,distance_h3
